First, we import SymEigen.

In [3]:
from SymEigen import *
from sympy import sqrt, Matrix

Say, we are calculating the Energy of a spring.
$$
E = \frac{1}{2} k (|\mathbf{x}-\mathbf{y}| - L_0)^2
$$
To compactly write the equation, we can define the following variables, fully 6 Dof:
$$
\mathbf{X} = 
\begin{bmatrix}
\mathbf{x} \\
\mathbf{y} 
\end{bmatrix}
$$
Then we can define such a matrix as follows:

In [4]:
X = Eigen.Vector('X', 6)
X

Matrix([
[X(0)],
[X(1)],
[X(2)],
[X(3)],
[X(4)],
[X(5)]])

Other coefficients are defined as follows:

In [5]:
k = Eigen.Scalar('k')
L0 = Eigen.Scalar('L0')

It's easy to calculate the Energy as follows:

In [6]:
X_l = Matrix(X[0:3])
X_r = Matrix(X[3:6])
d = X_l - X_r 

E = k * (sqrt(d.T * d) - L0)**2 / 2
E

Matrix([[k*(-L0 + sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2))**2/2]])

We use `VecDiff` to calculate the Vec/Vec derivative, so the Gradient of the Energy is:

In [7]:
G = VecDiff(E, X)
G

Matrix([
[ k*(-L0 + sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2))*(X(0) - X(3))/sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2)],
[ k*(-L0 + sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2))*(X(1) - X(4))/sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2)],
[ k*(-L0 + sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2))*(X(2) - X(5))/sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2)],
[k*(-L0 + sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2))*(-X(0) + X(3))/sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2)],
[k*(-L0 + sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2))*(-X(1) + X(4))/sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2)],
[k*(-L0 + sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2))*(-X(2) + X(5))/sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2)]])

So for the Hessian, we have:

In [8]:
H = VecDiff(G, X)
H

Matrix([
[ k*(-L0 + sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2))*(-X(0) + X(3))*(X(0) - X(3))/((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2)**(3/2) + k*(-L0 + sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2))/sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2) + k*(X(0) - X(3))**2/((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2),                                                                                                                             k*(-L0 + sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2))*(X(0) - X(3))*(-X(1) + X(4))/((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2)**(3/2) + k*(X(0) - X(3))*(X(1) - X(4))/((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2),                                                                                                                             k*(-L0 + sqrt((X(0) - X(3))**2 + (X(1) - X(4))**2 + (X(2) - X(5))**2))*(X(0) - X(3))*(-X(2) + X(5))/((X(0) - X

To generate Eigen Cpp code, we should do the following:
1. Declare a `EigenFunctionGenerator` as a context.
2. Wrap the Input Variable to a `Closure`.
3. Call the `Closure` by inputting the function name and `Expr` (e.g. the `E`, `G`, `H`).

In [9]:
Gen = EigenFunctionGenerator()
Gen.MacroBeforeFunction("__device__ __host__") # for CUDA
Closure = Gen.Closure(k, L0, X)

First, we generate the Eigen Cpp code for the Energy:

In [10]:
print(Closure('SpringEnergy', E, 'E'))

template <typename T>
__device__ __host__ void SpringEnergy(T& E, const T& k, const T& L0, const Eigen::Vector<T,6>& X)
{
/*****************************************************************************************************************************
Function generated by SymEigen.py 
Author: MuGdxy
GitHub: https://github.com/MuGdxy/SymEigen
E-Mail: lxy819469559@gmail.com
******************************************************************************************************************************
LaTeX expression:
//tex:$$E = \left[\begin{matrix}\frac{k \left(- L_{0} + \sqrt{\left(X(0) - X(3)\right)^{2} + \left(X(1) - X(4)\right)^{2} + \left(X(2) - X(5)\right)^{2}}\right)^{2}}{2}\end{matrix}\right]$$

Symbol Name Mapping:
k:
    -> {}
    -> Matrix([[k]])
L0:
    -> {}
    -> Matrix([[L0]])
X:
    -> {}
    -> Matrix([[X(0)], [X(1)], [X(2)], [X(3)], [X(4)], [X(5)]])
***************************************************************************************************************************

Then, Gradient:

In [11]:
print(Closure('SpringGradient', G, 'G'))

template <typename T>
__device__ __host__ void SpringGradient(Eigen::Vector<T,6>& G, const T& k, const T& L0, const Eigen::Vector<T,6>& X)
{
/*****************************************************************************************************************************
Function generated by SymEigen.py 
Author: MuGdxy
GitHub: https://github.com/MuGdxy/SymEigen
E-Mail: lxy819469559@gmail.com
******************************************************************************************************************************
LaTeX expression:
//tex:$$G = \left[\begin{matrix}\frac{k \left(- L_{0} + \sqrt{\left(X(0) - X(3)\right)^{2} + \left(X(1) - X(4)\right)^{2} + \left(X(2) - X(5)\right)^{2}}\right) \left(X(0) - X(3)\right)}{\sqrt{\left(X(0) - X(3)\right)^{2} + \left(X(1) - X(4)\right)^{2} + \left(X(2) - X(5)\right)^{2}}}\\\frac{k \left(- L_{0} + \sqrt{\left(X(0) - X(3)\right)^{2} + \left(X(1) - X(4)\right)^{2} + \left(X(2) - X(5)\right)^{2}}\right) \left(X(1) - X(4)\right)}{\sqrt{\left(X(0) - X(

Finally, Hessian:

In [12]:
print(Closure('SpringHessian', H, 'H'))

template <typename T>
__device__ __host__ void SpringHessian(Eigen::Matrix<T,6,6>& H, const T& k, const T& L0, const Eigen::Vector<T,6>& X)
{
/*****************************************************************************************************************************
Function generated by SymEigen.py 
Author: MuGdxy
GitHub: https://github.com/MuGdxy/SymEigen
E-Mail: lxy819469559@gmail.com
******************************************************************************************************************************
LaTeX expression:
//tex:$$H = \left[\begin{matrix}\frac{k \left(- L_{0} + \sqrt{\left(X(0) - X(3)\right)^{2} + \left(X(1) - X(4)\right)^{2} + \left(X(2) - X(5)\right)^{2}}\right) \left(- X(0) + X(3)\right) \left(X(0) - X(3)\right)}{\left(\left(X(0) - X(3)\right)^{2} + \left(X(1) - X(4)\right)^{2} + \left(X(2) - X(5)\right)^{2}\right)^{\frac{3}{2}}} + \frac{k \left(- L_{0} + \sqrt{\left(X(0) - X(3)\right)^{2} + \left(X(1) - X(4)\right)^{2} + \left(X(2) - X(5)\right)^{2}}\right